Le but de se code est de scrapper les prix littéraires en France entre 2019 et 2023

In [1]:
# on importe la bibliothèque requests
import requests
from bs4 import BeautifulSoup # on importe BeautifulSoup
import pandas as pd

On crée un dataframe vide avec comme colonnes les infos qu'on va importer

In [2]:
colonnes = ['Année', 'Prix', 'Auteur', 'Titre']

# Créer un DataFrame vide avec ces colonnes
df = pd.DataFrame(columns=colonnes)

print(df)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


On scrape

In [3]:
def scrape_wikipedia_page_france(year):
    global df
    url = f"https://fr.wikipedia.org/wiki/Prix_litt%C3%A9raires_{year}"
    response = requests.get(url)
    response.encoding = "utf-8"  # On garantit le bon encodage

    if response.status_code == 200:  # On vérifie si la requête est un succès
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # On recherche la section France
        france_section = soup.find('h2', string="France")
        if france_section:
            # La liste <ul> qui suit contient les prix pour la France
            france_list = france_section.find_next('ul')

            if france_list:
                # On recherche tous les <li> dans la liste
                prizes = france_list.find_all('li')

                # On extrait les données
                for prize in prizes:
                    # Nom du prix
                    prize_name = prize.find('a')
                    prize_name = prize_name.text.strip() if prize_name else None

                    # Auteur
                    author = prize.find_all('a')
                    author = author[1].text.strip() if len(author) > 1 else None

                    # Titre
                    title = prize.find('i')
                    title = title.text.strip() if title else None

                    # On ajoute les résultats si les données sont complètes
                    if prize_name and author and title:
                        new_row = {'Année': year, 'Prix': prize_name, 'Auteur': author, 'Titre': title}
                        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Section 'France' non trouvée pour l'année {year}")
    else:
        print(f"Erreur lors du scraping de l'année {year}: Status code {response.status_code}")

    return df

In [4]:
for year in range(2019, 2024):
    scrape_wikipedia_page_france(year)

In [5]:
df.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Par les routes,Par les routes
1,2019,Prix Femina étranger,Ordesa,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...


Création d'un fichier CSV afin de vérifier s'il n'y a pas d'incohérences

In [6]:
df.to_csv("prix_litteraires.csv", index=False)

PARTIE: Corriger les incohérences de la table

Création de fonctions

In [31]:
import requests
import re
from bs4 import BeautifulSoup as soup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_wikipedia(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)

    # URL de recherche sur Livraddict
    search_title = '%20'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.bing.com/search?q=Livre%20{search_title}%20{year}%20wikipedia&qs=n&form=QBRE&sp=-1&ghc=1&lq=0&pq=livre%20{lower_search_title}%20{year}%20wikipedia&sc=11-30&sk=&cvid=88409FB04E76408A93DE43207AD5027F&ghsh=0&ghacc=0&ghpl='

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response_text, "html.parser")
    first_result = page.find('li', class_='b_algo')  # Trouve le premier résultat
    if first_result:
        link = first_result.find('a', href=True)  # Trouve le lien à l'intérieur de <a>
        if link:
            href = link['href']
            # Si c'est une URL relative, complète-la
            if href.startswith('/wiki'):
                href = 'https://fr.wikipedia.org' + href
            return href
        else:
                print("Pas de lien trouvé dans le premier résultat.")
                return None
    else:
            print("Pas de résultats trouvés.")
            return None

Fonction pour récupérer l'information sur l'auteur depuis la page wikipedia

In [34]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).text
    if response == None:
        autor = "problème d'URL"
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    if table == None:
        autor = "Erreur de trouvaille de la table wikipédia" 
    else: 
        rows = table.find_all('tr')
        if rows == None:
            autor = "Pas de catégories trouvées dans la table"
        else:
            tr_auteur = find_tr_autor(rows)
            if tr_auteur == None:
                autor = "Pas de rubrique auteur dans la table"
            else:
                td_autor = tr_auteur.find("td")
                if td_autor == None:
                    autor = "Pas de nom dans la ligne auteur (td)"
                else:
                    a_autor = td_autor.find("a")
                    if a_autor == None :
                        autor = "Pas de nom dans la ligne auteur (a)"
                    else:
                        autor = a_autor.get("title")
                        if autor == None:
                            autor = "Pas de nom dans la ligne auteur (title)"
    return autor



In [32]:
def search_author(row):
    if row["Auteur"] == row["Titre"]:
        url = search_wikipedia("Titre", "Année")
        author = recup_info(url)
        return author
    else:
        return row["Auteur"]

Fonction globale allant chercher l'auteur de l'ouvrage quand ce dernier n'est pas indiqué

In [36]:
df['type_de_document'] = df.apply(search_author, axis=1)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


TEST POUR VOIR SI ca marche

In [43]:
search_wikipedia("La mer des monstres", "2006")

'https://fr.wikipedia.org/wiki/La_Mer_des_monstres'

In [17]:
recup_info('https://fr.wikipedia.org/wiki/La_Mer_des_monstres')

[<tr>
 <td class="entete universite italique" colspan="2" style="background-color:#7DA7D9;color:black;">La Mer des monstres<style data-mw-deduplicate="TemplateStyles:r160407116">.mw-parser-output .entete.universite{background-image:url("//upload.wikimedia.org/wikipedia/commons/4/42/Picto_infobox_book.png")}</style>
 </td></tr>,
 <tr><td colspan="2" style="display:none;">
 </td></tr>,
 <tr>
 <th scope="row">Auteur
 </th>
 <td><a href="/wiki/Rick_Riordan" title="Rick Riordan">Rick Riordan</a>
 </td>
 </tr>,
 <tr>
 <th scope="row">Pays
 </th>
 <td><span class="nowrap"><span class="datasortkey" data-sort-value="États unis"><span class="flagicon"><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description" href="/wiki/Fichier:Flag_of_the_United_States.svg" title="Drapeau des États-Unis"><img alt="Drapeau des États-Unis" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/th

In [16]:
def recup_info(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers).content
    page = BeautifulSoup(response, "html.parser")
    table = page.find('table', {'class': 'infobox'})
    rows = table.find_all('tr')
    return rows